<a href="https://colab.research.google.com/github/Kennethfargose/Time-Series-Analysis/blob/main/Time_series_forecasting_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pmdarima

In [ ]:
pip install fbprophet

In [ ]:
!pip install prophet

In [ ]:
import prophet


In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
from pmdarima.arima import auto_arima
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA, ARIMA

from statsmodels.tsa.statespace.sarimax import SARIMAX
# from fbprophet import prophet

from math import sqrt

import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
import seaborn as sns

from random import random

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error

In [ ]:


df = pd.read_csv('/content/drive/MyDrive/international-airline-passengers.csv')



In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/International_Report_Passengers.csv')

In [ ]:
df1

In [ ]:
df2=df1.drop(['data_dte','usg_apt_id','usg_apt','usg_wac','fg_apt_id','fg_apt','fg_wac','airlineid','carrier','type','Scheduled','Charter','carriergroup'],axis=1)



In [ ]:
df2['Year'].apply(str)

In [ ]:
c = np.concatenate(df2[["Year", "Month"]])

In [ ]:
df2


In [ ]:
df2.columns = ['year','passengers']

In [ ]:
df.columns = ['year','passengers']

df=df.dropna()

In [ ]:
df['year']=pd.to_datetime(df['year'])

In [ ]:
df.drop(df.tail(1).index,inplace=True)

In [ ]:
print('Shape of data',df.shape)
df.head()


In [ ]:


df.set_index('year',inplace=True)



In [ ]:

df.plot()



In [ ]:


test_result=adfuller(df['passengers'])



In [ ]:
from statsmodels.tsa.stattools import adfuller



#Ho: It is non stationary
#H1: It is stationary

def adfuller_test(passengers):
    result=adfuller(passengers)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")





In [ ]:

adfuller_test(df['passengers'])



In [ ]:


df['passengers First Difference'] = df['passengers'] - df['passengers'].shift(1)



In [ ]:


df['passengers'].shift(1)



In [ ]:
df['Seasonal First Difference']=df['passengers']-df['passengers'].shift(12)

In [ ]:
df.head(14)

In [ ]:
df.drop('Passengers First Difference',axis=1)

In [ ]:


## Again test dickey fuller test
adfuller_test(df['Seasonal First Difference'].dropna())



In [ ]:
df['Seasonal First Difference'].plot()

In [ ]:


from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df['passengers'])
plt.show()



In [ ]:


from statsmodels.graphics.tsaplots import plot_acf,plot_pacf



In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df['Seasonal First Difference'].iloc[13:],lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df['Seasonal First Difference'].iloc[13:],lags=40,ax=ax2)


In [ ]:
from pmdarima import auto_arima
# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:


model=sm.tsa.arima.ARIMA(df['passengers'],order=(1,1,1))
model_fit=model.fit()



In [ ]:
model_fit.summary()

In [ ]:
stepwise_fit = auto_arima(df['passengers'],
                          suppress_warnings=True)

stepwise_fit.summary()



In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
print(df.shape)
train=df.iloc[:-30]
test=df.iloc[-30:]
print(train.shape,test.shape)
print(test.iloc[0],test.iloc[-1])

In [ ]:
import statsmodels.api as sm
model=sm.tsa.arima.ARIMA(train['passengers'],order=(2,1,2))
model=model.fit()
model.summary()

In [ ]:
start=len(train)
end=len(train)+len(test)-1
#if the predicted values dont have date values as index, you will have to uncomment the following two commented lines to plot a graph
#index_future_dates=pd.date_range(start='2018-12-01',end='2018-12-30')
pred=model.predict(start=start,end=end,typ='levels').rename('ARIMA predictions')
#pred.index=index_future_dates
pred.plot(legend=True)
test['passengers'].plot(legend=True)

In [ ]:


pred.plot(legend='ARIMA Predictions')
test['passengers'].plot(legend=True)



In [ ]:


test['passengers'].mean()



In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(pred,test['passengers']))
print(rmse)

In [ ]:
model2=sm.tsa.arima.ARIMA(df['passengers'],order=(2,1,2))
model2=model2.fit()
df.tail()

In [ ]:
index_future_dates=pd.date_range(start='2018-12-30',end='2019-01-29')
#print(index_future_dates)
pred=model2.predict(start=len(df),end=len(df)+30,typ='levels').rename('ARIMA Predictions')
#print(comp_pred)
pred.index=index_future_dates
print(pred)


In [ ]:
pred.plot(figsize=(12,5),legend=True)


In [ ]:
model=sm.tsa.statespace.SARIMAX(df['passengers'],order=(1, 1, 1),seasonal_order=(1,1,1,12))
results=model.fit()

In [ ]:


df['forecast']=results.predict(start=120,end=190,dynamic=True)
df[['passengers','forecast']].plot(figsize=(12,8))



In [ ]:


from pandas.tseries.offsets import DateOffset
future_dates=[df.index[-1]+ DateOffset(months=x)for x in range(0,24)]



In [ ]:
future_datest_df=pd.DataFrame(index=future_dates[1:],columns=df.columns)

In [ ]:
future_datest_df.tail()

In [ ]:
future_df=pd.concat([df,future_datest_df])



In [ ]:


future_df['forecast'] = results.predict(start = 141, end = 165, dynamic= True)
future_df[['passengers', 'forecast']].plot(figsize=(12, 8))

